# 12. 用户查询太模糊？通过查询扩展，提升语义匹配能力

## 一、为什么要在 RAG 中进行 Query Expansion？

在构建基于 **RAG（Retrieval-Augmented Generation）** 的问答系统时，用户输入往往存在以下几个常见问题：

- 表达模糊、不完整或口语化  
- 缺乏上下文信息  
- 难以准确命中知识库中的相关文档  

如果直接使用用户的原始查询进行向量检索，可能会导致以下问题：

- 召回结果不足  
- 命中无关内容  
- 最终生成的答案不够准确或全面  

因此，在执行检索前对用户的问题进行 **语义级扩展与改写（Query Expansion / Rewriting）**，是一种有效的优化手段。

### 示例：模糊查询示例

面对如下模糊的用户提问：

- “我想去好玩的地方”  
- “有没有好吃的”  
- “适合亲子游的地方”  

这些问题缺乏具体的背景信息和明确的需求描述，直接检索往往难以命中关键内容。

### 解决方案

为了解决这一问题，可以采用以下两种策略来增强检索效果：

1. **问题改写（Query Rewriting）**  
   将模糊问题转化为更清晰、具体的问题，提升语义表达能力。

2. **多步骤检索（Multi-step Querying）**  
   将复杂问题拆解为多个子任务，逐步检索后整合答案，提高检索的全面性和准确性。

## 二 优化手段
### 2.1 在检索前进行问题改写（Query Rewriting）

安装依赖

In [ ]:
%pip install langchain faiss-cpu transformers torch sentence-transformers dashscope langchain-community "unstructured[md]"

加载文档并构建 FAISS 向量库

In [1]:
import logging
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models.tongyi import ChatTongyi

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 配置
DOCUMENT_PATH = './12/data/' 
FILE_PATTERN = '*.md'
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
# 通义Embedding模型
EMBEDDING_MODEL = 'text-embedding-v4'
LLM_MODEL = 'qwen-plus'  # 支持qwen-plus/qwen-max/qwen-turbo
TOP_K = 5

# 1. 初始化LLM（使用ChatTongyi）
logging.info(f"正在初始化{LLM_MODEL}模型...")
llm = ChatTongyi(
    model_name=LLM_MODEL,
    api_key=os.getenv("DASHSCOPE_API_KEY"),  # 使用DashScope API Key
    temperature=0.7,
    streaming=True  # 启用流式输出
)

# 2. 加载文档
logging.info(f"从{DOCUMENT_PATH}加载文档...")
loader = DirectoryLoader(DOCUMENT_PATH, glob=FILE_PATTERN)
docs = loader.load()
logging.info(f"成功加载{len(docs)}个文档")

# 3. 文档切片
logging.info("正在进行文档切片...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
texts = text_splitter.split_documents(docs)
logging.info(f"文档切片完成，获得{len(texts)}个文本块")

# 4. 初始化Embedding模型
logging.info(f"加载{EMBEDDING_MODEL}嵌入模型...")
embeddings = DashScopeEmbeddings(
        model=EMBEDDING_MODEL,
        dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
    )

# 5. 构建向量库
logging.info("构建FAISS向量库...")
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': TOP_K})

# 6. 创建RAG查询链
logging.info("创建RAG查询链...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

logging.info("RAG系统初始化完成，ready for query!")

# 添加查询示例
def query_rag(question):
    logging.info(f"处理查询: {question}")
    result = qa_chain.invoke(question)
    return result['result']



2025-07-16 15:04:19,837 - INFO - 正在初始化qwen-plus模型...
2025-07-16 15:04:20,005 - INFO - 从./12/data/加载文档...
2025-07-16 15:04:24,349 - INFO - 成功加载5个文档
2025-07-16 15:04:24,350 - INFO - 正在进行文档切片...
2025-07-16 15:04:24,350 - INFO - 文档切片完成，获得5个文本块
2025-07-16 15:04:24,351 - INFO - 加载text-embedding-v4嵌入模型...
2025-07-16 15:04:24,352 - INFO - 构建FAISS向量库...
2025-07-16 15:04:24,831 - INFO - Loading faiss.
2025-07-16 15:04:24,877 - INFO - Successfully loaded faiss.
2025-07-16 15:04:24,882 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
2025-07-16 15:04:24,883 - INFO - 创建RAG查询链...
2025-07-16 15:04:24,884 - INFO - RAG系统初始化完成，ready for query!


传统检索方式（无优化）

In [5]:
# 测试查询
if __name__ == "__main__":
    test_question = "我想去好玩的地方?"
    answer = query_rag(test_question)
    print(f"问题: {test_question}\n答案: {answer}")

2025-07-16 02:17:04,179 - INFO - 处理查询: 我想去好玩的地方?


问题: 我想去好玩的地方?
答案: 如果你喜欢好玩的地方，可以考虑以下推荐：

### 主题公园推荐
1. **上海迪士尼乐园（上海）**  
   - 适合亲子游、情侣、家庭出游。
   - 项目丰富，如“米奇童话专列”、“创极速光轮”、“飞跃地平线”等。
   - 有夜间灯光秀和丰富的表演。

2. **北京环球影城（北京）**  
   - 多种互动项目，适合全家游玩。
   - 有哈利波特、变形金刚等热门IP主题区。

3. **广州长隆欢乐世界（广州）**  
   - 动物园+游乐场+水上乐园一体化。
   - 适合带孩子的家庭，设施多样。

### 其他玩法推荐
- **亲子游**：除了主题公园，还可以去上海的海洋水族馆或者北京的动物园。
- **情侣游**：如果喜欢浪漫氛围，可以选择外滩夜景（上海）或新天地晚餐（上海）。
- **背包客**：可以尝试武康路骑行（上海）或探索衡山路的小众咖啡馆。

### 小贴士
- 建议错峰出行，避免节假日人流高峰。
- 提前查看景点开放时间和门票信息。

如果你告诉我具体的城市或兴趣，我可以提供更详细的推荐哦！


#### 2.1.1 使用 LLM 改写问题（增强语义表达）
Step 1: 我们首先定义一个 Prompt 模板，用于引导 LLM 对用户问题进行改写：

In [2]:
from langchain.prompts import PromptTemplate

rewrite_prompt = PromptTemplate.from_template(
"""
你是一个旅游助手，请将以下用户问题改写为更清晰、完整的形式：

原始问题：
{{question}}

请输出一个改写后的问题，要求：
- 更具体
- 包含旅行类型（亲子/情侣/自驾等）
- 能帮助系统准确检索相关信息

输出格式：
[改写后] - <你的回答>
""")

Step 2: 执行改写

In [3]:
from langchain.chains import LLMChain

# 初始化改写链
def create_rewrite_chain(llm): 
    return LLMChain(llm=llm, prompt=rewrite_prompt)

# 执行问题改写
def rewrite_question(rewrite_chain, question): 
    logging.info(f"正在改写问题: {question}")
    try:
        rewritten_question = rewrite_chain.run(question).strip()
        if not rewritten_question:
            logging.warning("改写后的问题为空")
            return "未获得有效的改写结果"
        return rewritten_question
    except Exception as e:
        logging.error(f"改写问题时出错: {str(e)}")
        return "改写问题时发生错误"



rewrite_chain = create_rewrite_chain(llm)
test_question = "我想去好玩的地方?"
rewritten = rewrite_question(rewrite_chain, test_question)
print(f"改写后问题: {rewritten}")

/var/folders/6l/bss6m08x2ssb9fhhyqwqrgwm0000gn/T/ipykernel_25067/596718747.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=rewrite_prompt)
2025-07-16 15:09:30,889 - INFO - 正在改写问题: 我想去好玩的地方?
/var/folders/6l/bss6m08x2ssb9fhhyqwqrgwm0000gn/T/ipykernel_25067/596718747.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rewritten_question = rewrite_chain.run(question).strip()
2025-07-16 15:09:30,890 - ERROR - 改写问题时出错: list index out of range


改写后问题: 改写问题时发生错误


In [11]:
answer = query_rag(rewritten)
print(f"问题: {rewritten}\n答案: {answer}")

2025-07-16 02:25:24,711 - INFO - 处理查询: [改写后] - 有没有适合亲子游的好玩地方推荐？


问题: [改写后] - 有没有适合亲子游的好玩地方推荐？
答案: 当然有！以下是几个适合亲子游的主题公园推荐：

1. **上海迪士尼乐园（上海）**  
   - 设施完善，儿童游乐区丰富。
   - 拥有多个适合孩子的项目，如“米奇童话专列”、“创极速光轮”等。
   - 提供婴儿车租赁、哺乳室和亲子餐厅等服务。

2. **北京环球影城（北京）**  
   - 多种互动项目，适合全家一起游玩。
   - 有丰富的角色互动和主题园区，让孩子体验沉浸式娱乐。

3. **广州长隆欢乐世界（广州）**  
   - 集动物园、游乐场、水上乐园于一体，适合不同年龄段的孩子。
   - 安全性高，并提供母婴设施。

这些主题公园的共同优点包括：提供婴儿车租赁服务、设有母婴室、以及安全性较高，非常适合带低龄儿童的家庭出游。

如果需要更详细的介绍或其他推荐，也可以告诉我哦！


检索结果更加聚焦于“亲子游”相关内容，有效提升了准确性。

### 2.2 多步骤检索（Multi-step Querying）

对于涉及多个需求的复杂问题，单一检索往往难以覆盖所有方面。此时，我们可以将其拆分为多个子问题，分别进行检索后再综合结果。

示例场景：查找适合亲子游的景点及周边美食

处理流程如下：
1. 先检索“亲子友好型景点”；
2. 再根据这些景点，检索“附近的推荐餐厅”；
3. 最后将两次检索结果整合，形成完整的回答。

这种分阶段检索的方式能够显著提升结果的准确性和全面性，尤其适用于涉及多个维度的复合型查询。

In [4]:
import logging
from langchain.agents import initialize_agent, Tool
from langchain.agents.types import AgentType

# 配置日志
def setup_logging(): 
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
setup_logging()

# 定义工具函数
def search_child_friendly_attractions(query): 
    try:
        logging.info(f"正在检索亲子友好型景点: {query}")
        result = qa_chain.run(query)
        if not result.strip(): 
            logging.warning("未检索到亲子友好型景点的结果")
            return "未找到相关亲子友好型景点"
        return result
    except Exception as e:
        logging.error(f"检索亲子友好型景点时出错: {str(e)}")
        return "检索亲子友好型景点时发生错误"

def search_nearby_restaurants(query): 
    try:
        logging.info(f"正在检索附近的推荐餐厅: {query}")
        result = qa_chain.run(query)
        if not result.strip(): 
            logging.warning("未检索到附近餐厅的结果")
            return "未找到附近的推荐餐厅"
        return result
    except Exception as e:
        logging.error(f"检索附近餐厅时出错: {str(e)}")
        return "检索附近餐厅时发生错误"

# 初始化工具
def initialize_tools(): 
    return [
        Tool(
            name="SearchChildFriendlyAttractions",
            func=search_child_friendly_attractions,
            description="用于检索亲子友好型景点，输入应该是与亲子游玩相关的问题"
        ),
        Tool(
            name="SearchNearbyRestaurants",
            func=search_nearby_restaurants,
            description="用于根据景点检索附近的推荐餐厅，输入应该包含景点名称"
        )
    ]

# 初始化代理
def initialize_search_agent(llm): 
    tools = initialize_tools()
    return initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 执行多步检索
def run_multi_step_search(agent, query): 
    try:
        logging.info(f"开始执行多步检索: {query}")
        response = agent.invoke(query)
        formatted_response = f"🔁 多步检索结果:\n{response['output']}"
        print(formatted_response)
        return formatted_response
    except Exception as e:
        logging.error(f"执行多步检索时出错: {str(e)}")
        print("执行多步检索时发生错误，请检查日志")
        return None

if __name__ == "__main__":
    try:
        if llm is None:
            logging.info("llm 变量未定义，正在使用默认 ChatOpenAI 进行初始化。")
            llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
        if qa_chain is None:
            logging.info("qa_chain 变量未定义，正在使用默认 LLMChain 进行初始化。")
            prompt_template = "请根据以下问题给出相关信息：{question}\n" \
                            "如果找不到相关信息，请明确说明 '未找到相关信息'。"
            qa_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

    except NameError:
        logging.error("llm 或 qa_chain 变量未定义，请确保在使用前正确初始化")
        raise

    agent = initialize_search_agent(llm)
    multi_step_query = """
    我想找一个适合带孩子玩的地方，附近最好有好吃的餐厅。
    先查有哪些亲子友好景点，再找附近的餐饮推荐。
    """
    run_multi_step_search(agent, multi_step_query)

/var/folders/6l/bss6m08x2ssb9fhhyqwqrgwm0000gn/T/ipykernel_25067/1306278018.py:53: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  return initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
2025-07-16 15:18:01,163 - INFO - 开始执行多步检索: 
    我想找一个适合带孩子玩的地方，附近最好有好吃的餐厅。
    先查有哪些亲子友好景点，再找附近的餐饮推荐。
    




> Entering new AgentExecutor chain...


2025-07-16 15:18:02,638 - INFO - 正在检索亲子友好型景点: 亲子游玩


我需要先查找亲子友好型景点，再根据景点查找附近的推荐餐厅。
Action: SearchChildFriendlyAttractions
Action Input: 亲子游玩
Observation: 如果你计划亲子游玩，以下是一些适合家庭带孩子的主题公园推荐：

### 1. 上海迪士尼乐园（上海）
- **特色**：中国大陆首座迪士尼主题公园，拥有多个主题园区，如“米奇童话专列”、“创极速光轮”、“飞跃地平线”等。
- **适合人群**：亲子游、情侣、家庭出游。
- **亲子友好设施**：
  - 婴儿车租赁服务
  - 哺乳室
  - 儿童游乐区
  - 亲子餐厅
- **推荐理由**：项目丰富，沉浸感强，是亲子游的首选目的地。
- **地址**：上海市浦东新区川沙镇

### 2. 北京环球影城（北京）
- **特色**：多种互动项目，适合全家一起体验，包括哈利波特、变形金刚等热门IP。
- **适合人群**：家庭出游。
- **亲子友好设施**：婴儿车租赁、母婴室。

### 3. 广州长隆欢乐世界（广州）
- **特色**：动物园+游乐场+水上乐园一体化，一站式满足亲子需求。
- **适合人群**：亲子家庭。
- **亲子友好设施**：婴儿车租赁、母婴室、安全性高的儿童区域。

### 旅行建议
- **错峰出行**：尽量避开节假日和周末高峰，减少排队时间。
- **关注最佳旅游时间**：
  - 上海：春秋季（3月-5月、9月-11月）为最佳季节。
  - 北京：春秋两季（4月-6月、9月-10月）最合适。
  - 桂林：4月-11月较宜，但需避开夏季暴雨高峰期。

如果需要更多具体信息，比如如何规划行程或周边配套景点，也可以告诉我哦！
Thought:

2025-07-16 15:18:18,596 - INFO - 正在检索附近的推荐餐厅: 上海迪士尼乐园


我需要根据推荐的亲子景点分别查找附近的推荐餐厅。首先选择其中一个景点，例如“上海迪士尼乐园”，然后检索其周边的餐饮推荐。

Action: SearchNearbyRestaurants
Action Input: 上海迪士尼乐园
Observation: 上海迪士尼乐园是中国大陆首座迪士尼主题公园，位于上海市浦东新区川沙镇。它是一个适合亲子游、情侣和家庭出游的理想目的地。

### 主题园区与项目
上海迪士尼乐园拥有多个精彩的主题园区，其中包括：
- **米奇童话专列**：充满童话色彩的巡游体验。
- **创极速光轮**：以《创》电影为背景的高速刺激过山车。
- **飞跃地平线**：身临其境的飞行体验项目，带你环游世界。

此外，园内还有丰富的表演活动和令人叹为观止的夜间灯光秀。

### 亲子友好设施
对于带孩子的家庭来说，乐园提供了许多便利设施，包括：
- 婴儿车租赁服务
- 哺乳室
- 儿童游乐区
- 亲子餐厅

### 推荐理由
上海迪士尼乐园是亲子游的首选地，不仅项目丰富，而且沉浸感强，非常适合家庭游玩。

### 旅行建议
- **错峰出行**：建议避开节假日或周末高峰时段，以减少排队时间。
- **最佳季节**：春秋季（3月-5月、9月-11月）气候宜人，是游览的最佳时间。

如果您需要更多关于交通、门票预订或其他细节的信息，请告诉我！
Thought:根据我的搜索，上海迪士尼乐园是一个非常适合亲子游玩的地方，同时乐园内和周边也有许多优质的亲子餐厅。以下是我对问题的总结回答：

Final Answer: 推荐您前往上海迪士尼乐园进行亲子游玩，这是一个沉浸感强、项目丰富的主题公园，拥有婴儿车租赁、哺乳室、儿童游乐区等亲子友好设施。园内还有多种亲子餐厅可供选择，例如“皇家宴会厅”、“帕帕里诺披萨”等，提供适合孩子的餐食。最佳游览时间为春秋季（3月-5月、9月-11月），建议避开节假日高峰以减少排队时间。

> Finished chain.
🔁 多步检索结果:
推荐您前往上海迪士尼乐园进行亲子游玩，这是一个沉浸感强、项目丰富的主题公园，拥有婴儿车租赁、哺乳室、儿童游乐区等亲子友好设施。园内还有多种亲子餐厅可供选择，例如“皇家宴会厅”、“帕帕里诺披萨”等，提供适合孩子的餐食。最佳游览时间为春秋季（3月-5月、9月-11月），建议避开节假日高峰以减少排队时

优势： 分阶段检索，提高准确性和全面性。

## 2.3 效果对比与评估

为了验证优化策略的有效性，建议构建一个小规模的 Ground Truth 数据集 ，并设计简单的评估指标进行对比分析。

评估方法示例：
- 构建包含原始问题、改写后问题、期望答案的数据集；
- 判断最终生成的回答是否包含正确答案；
- 统计准确率或召回率等指标。

In [6]:
import logging
from typing import List, Dict, Tuple
from difflib import SequenceMatcher

# 配置日志
def setup_logging(): 
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
setup_logging()

class GroundTruthDataset:
    """管理包含原始问题、改写后问题和期望答案的数据集"""
    def __init__(self):
        self.data: List[Dict[str, str]] = []

    def add_example(self, original_q: str, rewritten_q: str, ground_truth: str):
        """添加数据示例"""
        self.data.append({
            'original_q': original_q,
            'rewritten_q': rewritten_q,
            'ground_truth': ground_truth
        })
        logging.info("成功添加新的评估示例")

    def get_all_examples(self) -> List[Dict[str, str]]:
        """获取所有数据示例"""
        return self.data

class Evaluator:
    """评估模型回答的性能"""
    def __init__(self, similarity_threshold: float = 0.8):
        self.similarity_threshold = similarity_threshold

    def _is_answer_correct(self, answer: str, ground_truth: str) -> bool:
        """使用相似度判断答案是否正确"""
        similarity = SequenceMatcher(None, answer.lower(), ground_truth.lower()).ratio()
        return similarity >= self.similarity_threshold

    def evaluate_single(self, original_q: str, rewritten_q: str, answer: str, ground_truth: str) -> Tuple[int, Dict[str, str]]:
        """评估单个示例"""
        score = 1 if self._is_answer_correct(answer, ground_truth) else 0
        result = {
            'original_q': original_q,
            'rewritten_q': rewritten_q,
            'answer': answer,
            'ground_truth': ground_truth,
            'is_correct': score == 1
        }
        logging.info(f"原始问题: {original_q}\n改写问题: {rewritten_q}\n正确答案包含: {'✅' if score else '❌'}")
        return score, result

    def evaluate_dataset(self, dataset: GroundTruthDataset, answer_generator) -> Dict[str, float]:
        """评估整个数据集"""
        total = len(dataset.get_all_examples())
        correct = 0
        results = []

        for example in dataset.get_all_examples():
            try:
                answer = answer_generator(example['rewritten_q'])
                score, result = self.evaluate_single(
                    example['original_q'],
                    example['rewritten_q'],
                    answer,
                    example['ground_truth']
                )
                correct += score
                results.append(result)
            except Exception as e:
                logging.error(f"评估示例时出错: {str(e)}")
                continue

        accuracy = correct / total if total > 0 else 0
        # 可以扩展计算召回率、精确率、F1值等指标
        return {
            'accuracy': accuracy,
            'correct_count': correct,
            'total_count': total,
            'results': results
        }

    def generate_report(self, metrics: Dict[str, float]) -> str:
        """生成评估报告"""
        report = f"评估报告:\n"
        report += f"- 总样本数: {metrics['total_count']}\n"
        report += f"- 正确样本数: {metrics['correct_count']}\n"
        report += f"- 准确率: {metrics['accuracy']:.2%}\n"
        return report

# 示例使用
if __name__ == "__main__":
    # 初始化数据集
    dataset = GroundTruthDataset()
    dataset.add_example(
        original_q="我想带孩子玩",
        rewritten_q="推荐适合亲子游玩的目的地",
        ground_truth="推荐了上海迪士尼乐园和北京环球影城两个亲子游目的地"
    )

    # 模拟回答生成函数（实际使用时替换为真实的模型调用）
    def mock_answer_generator(query: str) -> str:
        if "亲子游玩" in query:
            return "推荐了上海迪士尼乐园和北京环球影城两个亲子游目的地"
        return "未找到相关信息"

    # 初始化评估器
    evaluator = Evaluator(similarity_threshold=0.8)

    # 执行评估
    metrics = evaluator.evaluate_dataset(dataset, mock_answer_generator)

    # 生成并打印评估报告
    report = evaluator.generate_report(metrics)
    print(report)

2025-07-16 15:21:51,627 - INFO - 成功添加新的评估示例
2025-07-16 15:21:51,628 - INFO - 原始问题: 我想带孩子玩
改写问题: 推荐适合亲子游玩的目的地
正确答案包含: ✅


评估报告:
- 总样本数: 1
- 正确样本数: 1
- 准确率: 100.00%



通过此类评估，可以量化不同策略的效果差异，为后续优化提供数据支持。

### 总结

| 优化点 | 效果 |
| --- | --- |
| 问题改写 | 提高检索准确性，避免遗漏关键信息 |
| 多步骤检索 | 提升复杂问题的回答完整性 |
| 中文 Embedding | 提升向量召回质量 |
| 本地 LLM | 隐私安全，可控性强 |


### 三、更多优化建议

| 优化方法 | 收益点 |
| --- | --- |
| 多变体召回 + 聚合 | 将多个改写后的查询同时提交给向量库，聚合结果 |
| HyDE 策略 | 利用假设文档（Hypothetical Document）辅助检索 |
| 多路召回 + Reranking | 使用 rerank 模块对检索结果打分排序，提升精度 |
